In [ ]:
#| default_exp foundation.codeformat

# Code Formatting
> Tests to check code formatting and quality

Using the `setup.cfg` in this project clean up code cells

In [ ]:
#| export
import black
import itertools
import re
import subprocess
from tokenize import tokenize,COMMENT
from ast import parse,FunctionDef
from io import BytesIO
from textwrap import dedent
from types import SimpleNamespace
from inspect import getsource,isfunction,isclass,signature,Parameter
from fastcore.basics import *
import re

In [ ]:
#| export
BLACK_OPTIONS = {
    "--line-length": str(119),
}

In [ ]:
#| export
def format_black(
    input_code:str # Input code to be passed through to black
) -> str:
    """
    Converts `code` into the properly-formatted `black` options
    """
    output_code = subprocess.run(
        ["black", "--code", input_code] + list(sum(BLACK_OPTIONS.items(), ()))
        ,
        capture_output=True
    )
    return output_code.stdout.decode()

In [ ]:
#| example
format_black("print ( 'hello world' )")

'print("hello world")\n'

In [ ]:
#| export
def _parses(s):
    """Parse Python code in string or function object `s`"""
    return parse(dedent(getsource(s) if isfunction(s) else s))

In [ ]:
#| export
def _tokens(s):
    """Tokenize Python code in string or function object `s`"""
    if isfunction(s): s = getsource(s)
    return tokenize(BytesIO(s.encode('utf-8')).readline)

In [ ]:
#| export
_clean_re = re.compile('^\s*#(.*)\s*$')

In [ ]:
#| export
def _clean_comment(s):
    """a"""
    res = _clean_re.findall(s)
    return res[0] if res else None

In [ ]:
#| export
def _param_locs(s, returns=True):
    """`dict` of parameter line numbers to names"""
    body = _parses(s).body
    if len(body)!= 1 or not isinstance(body[0], FunctionDef): return None
    defn = body[0]
    res = {arg.lineno:arg.arg for arg in defn.args.args}
    if returns and defn.returns: res[defn.returns.lineno] = 'return'
    return res

In [ ]:
#| export
def _get_comment(line, arg, comments, parms):
    """a"""
    if line in comments: return comments[line].strip()
    line -= 1
    res = []
    while line and line in comments and line not in parms:
        res.append(comments[line])
        line -= 1
    return dedent('\n'.join(reversed(res))) if res else None

In [ ]:
#| export
def _get_full(anno, name, default, docs):
    """a"""
    if anno==Parameter.empty and default!=Parameter.empty: anno = type(default)
    return AttrDict(docment=docs.get(name), anno=anno, default=default)

In [ ]:
#| export
def docments(s, full=False, returns=True, eval_str=False):
    """`dict` of parameter names to 'docment-style' comments in function or string `s`"""
    if isclass(s): s = s.__init__ # Constructor for a class
    comments = {o.start[0]:_clean_comment(o.string) for o in _tokens(s) if o.type==COMMENT}
    parms = _param_locs(s, returns=returns)
    docs = {arg:_get_comment(line, arg, comments, parms) for line,arg in parms.items()}
    if not full: return docs

    if isinstance(s,str): s = eval(s)
    sig = signature(s)
    res = {arg:_get_full(p.annotation, p.name, p.default, docs) for arg,p in sig.parameters.items()}
    if returns: res['return'] = _get_full(sig.return_annotation, 'return', Parameter.empty, docs)
    if eval_str:
        hints = type_hints(s)
        for k,v in res.items():
            if k in hints: v['anno'] = hints.get(k)
    return res

In [ ]:
code = '''def attach_example(
    source_code:str, # The complete python code of a function or class
    example_code:str # The complete annotated example to go along with the source code
) -> str:
    """
    Attaches example code to source code in a code cell
    """
    example_code = example_code.split("")
    
    num_spaces = sum(1 for _ in itertools.takewhile(str.isspace, example_code[0]))
'''

In [ ]:
#| export
def get_annotations(code:str):
    """
    Finds the annotations in some source code
    """
    annotations = {}
    tokens = list(_tokens(code))
    for i,token in enumerate(tokens):
        for name in docments(code, returns=False).keys():
            if name not in annotations.keys():
                if token.string == name and tokens[i+1].string == ":":
                    annotations[name] = tokens[i+2].string
        if token.string == "->":
            annotations["return"] = tokens[i+1].string
    return annotations

In [ ]:
get_annotations(code)

{'source_code': 'str', 'example_code': 'str', 'return': 'str'}

In [ ]:
#| export
def reformat_annotation_docstring(
    source_code:str, # The complete python code of a function or class
) -> str:
    """
    Takes the docstring from `source_code` and formats its docment inputs to 
    Hugging Face/Google style.
    """
    function_docments = docments(source_code)
    if all(documentation == None for documentation in function_docments.values()):
        return source_code
    function_annotations = get_annotations(source_code)
    beginning, docstring, end = source_code.split('\"\"\"')
    search = re.search("^(\s*)(?:\S|$)", docstring.lstrip("\n"))
    if search is None:
        num_spaces = 0
    else:
        num_spaces = len(search.groups()[0])
    spaces = " " * num_spaces
    if not docstring.startswith("\n"):
        docstring = f"\n{spaces}{docstring}\n"
    docstring += "\n" + spaces + "Args:"
    for variable in function_docments.keys():
        beginning = beginning.replace(f"# {function_docments[variable]}", "")
        if variable != "return":
            docstring += f'\n{spaces}  {variable} (`{function_annotations.get(variable, "Any")}`)'
            if function_docments[variable] != None:
                docstring += f':\n{spaces}    {function_docments[variable]}'
        elif function_docments[variable] != None:
            docstring += f'\n{spaces}Returns: {function_docments[variable]}'
    docstring += f"\n{spaces}"
    return '\"\"\"'.join((beginning, docstring, end))

In [ ]:
print(reformat_annotation_docstring(code))

def attach_example(
    source_code:str, 
    example_code:str 
) -> str:
    """
    Attaches example code to source code in a code cell
    
    Arguments:
      source_code (`str`):
        The complete python code of a function or class
      example_code (`str`):
        The complete annotated example to go along with the source code
    """
    example_code = example_code.split("")
    
    num_spaces = sum(1 for _ in itertools.takewhile(str.isspace, example_code[0]))



In [ ]:
example = """attach_example(
  source_code = "some code",
  example_code = "my example"
)"""

In [ ]:
#| export
def attach_example(
    source_code:str, # The complete python code of a function or class
    example_code:str # The complete annotated example to go along with the source code
) -> str:
    """
    Attaches example code to source code in a code cell
    """
    beginning, docstring, end = source_code.split('\"\"\"')
    search = re.search("^(\s*)(?:\S|$)", docstring.split("\n")[0].lstrip("\n"))
    if search is None:
        num_spaces = 4
    else:
        num_spaces = len(search.groups()[0]) + 4
    spaces = " " * num_spaces
    docstring += f"\n{spaces}Example:\n\n{spaces}```python\n"
    example_code = example_code.split("\n")
    for i,line in enumerate(example_code):
        if line[0].isspace() or line[0] in [")", "}"]:
            example_code[i] = f'{spaces}... {line}'
        else:
            example_code[i] = f'{spaces}>>> {line}'
    example_code += [f"{spaces}```\n{spaces}"]
    docstring += '\n'.join(example_code)
    return '\"\"\"'.join((beginning, docstring, end))

In [ ]:
#| example
attach_example(
  source_code = "some code",
  example_code = "my example"
)

In [ ]:
print(attach_example(reformat_annotation_docstring(code), example))

def attach_example(
    source_code:str, 
    example_code:str 
) -> str:
    """
    Attaches example code to source code in a code cell
    
    Arguments:
      source_code (`str`):
        The complete python code of a function or class
      example_code (`str`):
        The complete annotated example to go along with the source code
    
    Example:

    ```python
    >>> attach_example(
    ...   source_code = "some code",
    ...   example_code = "my example"
    ... )
    ```
    """
    example_code = example_code.split("")
    
    num_spaces = sum(1 for _ in itertools.takewhile(str.isspace, example_code[0]))



In [ ]:
#| export
from nbdev.processors import Processor
from nbdev.process import extract_directives

In [ ]:
#| export
class CodeCleaner(Processor):
    def cell(self, cell):
        if cell.cell_type == "code" and "export" in cell.directives_ and "import" not in cell.source:
            if ("def " in cell.source) or ("class " in cell.source):
                cell.source = reformat_annotation_docstring(
                    cell.source,
                )

In [ ]:
#| export
class DocstringExampleAdder(Processor):
    has_example = False
    previous_cell = None
    def cell(self, cell):
        if cell.cell_type == "code":
            if "export" in cell.directives_:
                if "import" not in cell.source and ("def " in cell.source or "class " in cell.source):
                    self.previous_cell = cell
            if "example" in cell.directives_:
                self.previous_cell.source = attach_example(
                    self.previous_cell.source, 
                    cell.source
                )
                self.previous_cell = None
                self.nb.cells.remove(cell)

In [ ]:
from nbdev.processors import mk_cell, NBProcessor, dict2nb

In [ ]:
nb = {
    "cells":[
    mk_cell(f'#| export\n{code}', "code"),
    mk_cell("#| example\n" + example, "code"),
]}

In [ ]:
processor = NBProcessor(procs=[CodeCleaner, DocstringExampleAdder], nb=dict2nb(nb))

In [ ]:
processor.process()

In [ ]:
print(processor.nb["cells"][0].source)

def attach_example(
    source_code:str, 
    example_code:str 
) -> str:
    """
    Attaches example code to source code in a code cell
    
    Arguments:
      source_code (`str`):
        The complete python code of a function or class
      example_code (`str`):
        The complete annotated example to go along with the source code
    
    Example:

    ```python
    >>> attach_example(
    ...   source_code = "some code",
    ...   example_code = "my example"
    ... )
    ```
    """
    example_code = example_code.split("")
    
    num_spaces = sum(1 for _ in itertools.takewhile(str.isspace, example_code[0]))

